In [ ]:
# merging feather files of kriging output
# doing calibration

In [1]:
"""
Script for Merging and Calibrating InSAR Displacement Data

This script processes InSAR displacement data by merging feather files, extracting
common displacement values, and calibrating the data using back-transformed values.

Author: [Your Name]
Date: [Today's Date]
"""

from appgeopy import *
from my_packages import *

# ------------------------------------------------------------------------------
# Script Information
# ------------------------------------------------------------------------------
print("\n\n" + "-" * 50)
print("Script: Merge and Calibrate InSAR Displacement Data")
print("-" * 50)

# ------------------------------------------------------------------------------
# Utility Functions
# ------------------------------------------------------------------------------
def log_time(start_time):
    """Log the elapsed time since start."""
    end = time.time()
    elapsed_time_seconds = round((end - start_time), 2)
    elapsed_time_minutes = round((elapsed_time_seconds / 60), 2)
    elapsed_time_hours = round((elapsed_time_seconds / 3600), 2)

    print(f"Processing time (seconds): {elapsed_time_seconds}")
    print(f"Processing time (minutes): {elapsed_time_minutes}")
    print(f"Processing time (hours): {elapsed_time_hours}")


# ------------------------------------------------------------------------------
# Main Functionality
# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------
# Configuration File Reading and Setup
# ------------------------------------------------------------------------------
config_fpath = r"D:\VENV_PYTHON\insar_scriptflow\D__merged_post_insstackpsi.ini"
config = configparser.ConfigParser()
config.read(config_fpath)



--------------------------------------------------
Script: Merge and Calibrate InSAR Displacement Data
--------------------------------------------------


['D:\\VENV_PYTHON\\insar_scriptflow\\D__merged_post_insstackpsi.ini']

In [ ]:
start = time.time()

# Extract general parameters
workflow_step = config["GENERAL"]["workflow_step"]
main_folder = config["GENERAL"]["insstackpsi_mainfolder"]

if workflow_step != "calibration":
    sys.exit(
        "'{}' is not for this program. Must be 'evaluation'".format(
            workflow_step
        )
    )

post_insstackpsi_fld = os.path.join(main_folder, "14_POST-PROCESSING")

# Setup folder paths
calibration_folder = os.path.join(main_folder, "15_CALIBRATION")
kriging_main_folder = os.path.join(
    calibration_folder, "1__KrigingInterpolation"
)
kriging_folder_number = config["POSTKRIGING"]["kriging_folder_number"]

# ------------------------------------------------------------------------------
# Identify Source Folders and Files
# ------------------------------------------------------------------------------
source_folders = glob(
    os.path.join(kriging_main_folder, "*LOS*", kriging_folder_number, "GAShp*")
)
select_source_fld = source_folders[0]
available_feather_files = glob(
    os.path.join(select_source_fld, "BACKTRANS*.feather")
)

available_feather_files

In [ ]:
# ------------------------------------------------------------------------------
# Merge Feather Files
# ------------------------------------------------------------------------------
merged_feather = pd.concat(
    [pd.read_feather(f) for f in available_feather_files], ignore_index=True
)
merged_feather = (
    merged_feather.drop_duplicates(subset="PointKey", keep="first")
    .sort_values(by="PointKey")
    .reset_index(drop=True)
)
print(f"Merged feather file shape: {merged_feather.shape}")

In [ ]:
# ------------------------------------------------------------------------------
# Load Raw Displacement Data
# ------------------------------------------------------------------------------
o_displacement_fpath = glob(
    os.path.join(post_insstackpsi_fld, "ras2pnt*MODIFIED*DISPLACEMENT*.pkl")
)[0]
o_displacement_df = pd.read_pickle(o_displacement_fpath, compression="zip")
print(f"Raw displacement file shape: {o_displacement_df.shape}")

In [ ]:
# ------------------------------------------------------------------------------
# Extract Common Displacement Data
# ------------------------------------------------------------------------------
common_columns = o_displacement_df.columns.intersection(merged_feather.columns)
extracted_displacement_df = (
    o_displacement_df.query("PointKey in @merged_feather.PointKey")[
        common_columns
    ]
    .sort_values(by="PointKey")
    .reset_index(drop=True)
)
print(f"Extracted displacement file shape: {extracted_displacement_df.shape}")

# Ensure PointKey columns match
if not extracted_displacement_df["PointKey"].equals(merged_feather["PointKey"]):
    sys.exit(
        "Error: PointKey columns do not match between extracted and merged data."
    )

In [ ]:
# ------------------------------------------------------------------------------
# Calibrate Displacement Data
# ------------------------------------------------------------------------------
measurement_columns = [col for col in common_columns if col.startswith("N")]
calibrated_displacement_df = extracted_displacement_df.copy()
calibrated_displacement_df[measurement_columns] = (
    calibrated_displacement_df[measurement_columns]
    + merged_feather[measurement_columns]
)
print(f"Calibrated displacement file shape: {calibrated_displacement_df.shape}")

# ------------------------------------------------------------------------------
# Performance Logging
# ------------------------------------------------------------------------------
log_time(start)

In [ ]:
calibrated_displacement_df.iloc[:, 3:]

In [ ]:
calibrated_cdisp_df = pd.concat(
    [
        calibrated_displacement_df.iloc[:, :3],
        calibrated_displacement_df.iloc[:, 3:].cumsum(axis=1),
    ]
)

In [2]:
df1 = pd.read_pickle(
    r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\14_POST-PROCESSING\ras2pnt_MODIFIED_oChoushui_CUMDISP_LOS_mm.pkl",
    compression="zip",
)
df1.shape

(3913806, 66)

In [6]:
df2 = pd.read_pickle(
    r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\16_VALIDATION\LOS_output_002\CALIBRATED_oChoushui_DISPLACEMENT_LOS_mm.pkl",
    compression="zip",
)
df2.shape

(142724, 56)

In [7]:
df1.head(5)

,PointKey,X_TWD97,Y_TWD97,N20220407,N20220419,N20220501,N20220513,N20220525,N20220606,N20220618,...,N20240115,N20240127,N20240208,N20240220,N20240303,N20240315,N20240327,N20240408,N20240420,geometry
0,X237393Y2705479,237393.759851,2.705479e+06,-7.6146,-9.6779,-1.5103,13.2467,4.5566,5.2099,10.6543,...,67.6839,50.4384,45.2853,65.8027,74.2053,75.3534,77.9708,81.4065,94.7549,POINT (237393.760 2705479.209)
1,X237382Y2705465,237382.784009,2.705465e+06,-7.3033,-9.4580,-1.2588,13.4363,4.8757,5.0949,10.3587,...,66.5557,49.2441,44.0067,64.5401,72.6544,74.1423,76.8493,80.4706,93.8391,POINT (237382.784 2705465.080)
2,X237405Y2705465,237405.139575,2.705465e+06,-7.4136,-9.5174,-1.1628,13.4627,4.9123,4.9551,10.9226,...,68.7627,51.4178,46.3779,66.9026,75.6598,76.5272,79.0719,82.4995,95.9623,POINT (237405.140 2705465.403)
3,X237450Y2705438,237450.254591,2.705438e+06,-7.7303,-9.9102,-1.6172,13.2474,4.3328,3.5675,9.9899,...,63.0528,45.2381,40.2910,60.8554,69.2934,70.5222,72.4235,76.0471,89.3224,POINT (237450.255 2705438.114)
4,X237472Y2705438,237472.610164,2.705438e+06,-7.3976,-9.7601,-1.2435,13.4520,5.1330,4.3768,11.3183,...,66.5965,48.2814,43.5823,64.3737,72.2750,73.5149,75.0915,78.6874,91.9702,POINT (237472.610 2705438.437)


In [8]:
df2.head(3)

,PointKey,X_TWD97,Y_TWD97,N20220407,N20220419,N20220501,N20220513,N20220525,N20220606,N20220618,...,N20230917,N20230929,N20231011,N20231023,N20231104,N20231116,N20231128,N20231210,N20231222,geometry
0,X166721Y2605093,166721.712260,2.605093e+06,-7.169121,-0.407133,-0.000448,2.661185,5.670715,4.550673,0.473342,...,1.773235,1.247375,8.677636,-3.763469,-1.566519,-1.127951,-4.920170,4.121250,-14.305912,POINT (166721.712 2605093.000)
1,X166723Y2604995,166723.076599,2.604995e+06,-7.542090,-0.693322,0.020005,3.508139,4.658548,4.420939,0.219339,...,-0.258695,1.357537,8.228381,-3.034549,-2.307993,-1.162280,-5.293896,4.310677,-14.014431,POINT (166723.077 2604995.262)
2,X166731Y2605176,166731.716579,2.605177e+06,-7.223977,-0.800864,-0.283953,2.511603,5.767942,5.114224,-1.355217,...,2.792092,0.049806,7.151204,-3.195049,-3.119060,-0.669251,-5.865661,3.898081,-14.138118,POINT (166731.717 2605176.932)
